In [1]:
import numpy as np
import pandas as pd
from sklearn.inspection import permutation_importance
from sklearn.ensemble import RandomForestClassifier
from base_module import (
    get_preprocessed_data,
    train_and_evaluate,
    save_model_results,
    SVM_PARAMS,
    RANDOM_STATE
)
import time

ALGORITHM_NAME = "Permutation_Importance_RF"
# !! IMPORTANT: Update this path to your local file path !!
DATA_PATH = r'D:\ACTUAL STUDY MATERIAL\IPD\Data\diabetic_data.csv'

print("="*70)
print(f"FEATURE SELECTION: {ALGORITHM_NAME}")
print("="*70)

FEATURE SELECTION: Permutation_Importance_RF


In [2]:
print("\n--- Loading and Preprocessing Data ---")

X_train_encoded, X_test_encoded, y_train, y_test, \
    all_encoded_features, all_original_features, feature_mapping = \
    get_preprocessed_data(DATA_PATH)

print(f"\nStarting with {len(all_encoded_features)} encoded features")


--- Loading and Preprocessing Data ---
Loading data from D:\ACTUAL STUDY MATERIAL\IPD\Data\diabetic_data.csv...


d:\ACTUAL STUDY MATERIAL\IPD\src\Notebook\base_module.py:32: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, na_values='?')


Cleaned data shape: (98053, 43)
Engineering features...
Engineered data shape: (98053, 41)

Preprocessing data...
Ordinal Encoding 31 features...
Encoded features: 40
  - Numeric: 9
  - Categorical (Ordinal): 31

Starting with 40 encoded features


In [3]:
print(f"\n--- Applying {ALGORITHM_NAME} (Sklearn Implementation) ---")

# --- 1. Train a model to get importances from ---
# This model is ONLY for calculating importances, not final evaluation.
# RandomForest is a good, standard choice.
estimator = RandomForestClassifier(
    n_estimators=100,
    random_state=RANDOM_STATE,
    n_jobs=-1
)

print("Fitting RandomForest model to calculate permutation importances...")
start_time = time.time()
estimator.fit(X_train_encoded, y_train)
elapsed = (time.time() - start_time) / 60
print(f"✓ Model fitting complete in {elapsed:.2f} minutes")

# --- 2. Calculate Permutation Importance ---
# We calculate importance on the *test* set (or a validation set)
# to see which features generalize best.
# 'f1' is a good scoring metric for imbalanced data.
print("\nCalculating permutation importances (this may take a minute)...")
start_time = time.time()
perm_importance = permutation_importance(
    estimator,
    X_test_encoded,  # Use test set for unbiased importance
    y_test,
    n_repeats=10,
    random_state=RANDOM_STATE,
    scoring='f1',
    n_jobs=-1
)
elapsed = (time.time() - start_time) / 60
print(f"✓ Permutation importance calculated in {elapsed:.2f} minutes")

# --- 3. Rank and Select Features ---
importances = perm_importance.importances_mean
importances_df = pd.DataFrame({
    'feature': all_encoded_features,
    'importance': importances
}).sort_values(by='importance', ascending=False)

# Select top 70% of features (like your FIFA notebook)
n_select = int(0.7 * len(all_encoded_features))
selected_features = importances_df['feature'].head(n_select).tolist()

# --- 4. Filter Datasets ---
# Get the column indices in the *original* encoded data
original_indices_mapping = {feat: i for i, feat in enumerate(all_encoded_features)}
final_selected_original_indices = [original_indices_mapping[feat] for feat in selected_features]

X_train_selected = X_train_encoded[:, final_selected_original_indices]
X_test_selected = X_test_encoded[:, final_selected_original_indices]

print(f"\nFeature Selection Results:")
print(f"  Selected: {len(selected_features)} features")
print(f"  Eliminated: {len(all_encoded_features) - len(selected_features)} features")


--- Applying Permutation_Importance_RF (Sklearn Implementation) ---
Fitting RandomForest model to calculate permutation importances...
✓ Model fitting complete in 0.02 minutes

Calculating permutation importances (this may take a minute)...
✓ Permutation importance calculated in 0.55 minutes

Feature Selection Results:
  Selected: 28 features
  Eliminated: 12 features


In [4]:
print(f"\n--- Training Model on {ALGORITHM_NAME}-Selected Features ---")

clf, y_pred, report = train_and_evaluate(
    X_train_selected, X_test_selected,
    y_train, y_test,
    **SVM_PARAMS
)

print(f"\nModel Performance:")
print(f"  Accuracy: {report['accuracy']:.4f}")
print(f"  Precision (class 0): {report['0']['precision']:.4f}")
print(f"  Recall (class 0): {report['0']['recall']:.4f}")
print(f"  F1-score (class 0): {report['0']['f1-score']:.4f}")
print(f"  Precision (class 1): {report['1']['precision']:.4f}")
print(f"  Recall (class 1): {report['1']['recall']:.4f}")
print(f"  F1-score (class 1): {report['1']['f1-score']:.4f}")


--- Training Model on Permutation_Importance_RF-Selected Features ---

Training SVM classifier...


d:\ACTUAL STUDY MATERIAL\IPD\.venv\Lib\site-packages\sklearn\svm\_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Evaluating...

Model Performance:
  Accuracy: 0.5978
  Precision (class 0): 0.9065
  Recall (class 0): 0.6095
  F1-score (class 0): 0.7289
  Precision (class 1): 0.1414
  Recall (class 1): 0.5056
  F1-score (class 1): 0.2210


In [5]:
print(f"\n--- Saving Results ---")

save_model_results(
    algorithm_name=ALGORITHM_NAME,
    selected_encoded_features=selected_features,
    all_encoded_features=all_encoded_features,
    all_original_features=all_original_features,
    feature_mapping=feature_mapping,
    report=report,
    results_csv='study_results.csv'
)

print("\n" + "="*70)
print(f"{ALGORITHM_NAME} COMPLETE")
print("="*70)


--- Saving Results ---

Saving results for: Permutation_Importance_RF
✓ Appended to D:\ACTUAL STUDY MATERIAL\IPD\src\Results\study_results.csv
✓ Saved to D:\ACTUAL STUDY MATERIAL\IPD\src\Results\permutation_importance_rf_results.json

  Original: 28/40 features
  Encoded:  28/40 features
  Accuracy: 0.5978
  F1 (class 1): 0.2210

Permutation_Importance_RF COMPLETE
